# 6.8701 PSET 3

**Deadline: Tues Nov 12th 11:59pm**

In this homework, you will get some practical experience with modeling small molecules, docking, and understanding eQTLs. The goal is to expose you to useful libraries (torch, rdkit, dgl) that are heavily utilized in practice, understand some of the state of the art research in structural biology, and practice with genetic association studies by studying eQTLs.

The questions are designed in a way that you should not need to know these libraries to answer them, but if anything is confusing or if you get stuck please ask for help on Piazza!

Answer all questions directly in this notebook and complete the missing code where marked with **COMPLETE HERE**.

## 0. Setup

Run the cell below to import all the relevant libraries.

In [ ]:
!pip install  dgl -f https://data.dgl.ai/wheels/torch-2.3/repo.html # errors happen with torch==2.5.0
!pip install matplotlib
!pip install scikit-learn
!pip install torch==2.3.1 # errors happen with torch==2.5.0
!pip install rdkit
!pip install dgllife
!pip install tqdm
!pip install scikit-learn
!pip install scipy
!pip install numpy
!pip install scanpy
!pip install leidenalg

import torch
import numpy as np
from sklearn.datasets import make_classification
from sklearn.datasets import make_gaussian_quantiles
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from dgllife.data import Tox21
from dgllife.utils import SMILESToBigraph, CanonicalAtomFeaturizer, CanonicalBondFeaturizer
from rdkit import Chem
from rdkit.Chem import AllChem
from tqdm import tqdm
import os
import dgl

import scanpy as sc
import anndata
import pandas as pd
import scipy.io as io
from google.colab import files
from scipy.spatial.transform import Rotation

Looking in links: https://data.dgl.ai/wheels/torch-2.3/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


## 1. Introduction to Neural Networks

In the previous problem set (PSET 2), we implemented a simple logistic regression model and an active learning-based model. In this PSET, we will be working with more complex models such as RNNs (LSTMs) and GNNs, where a strong technical understanding is necessary. This section will go more in-depth on the fundamentals.

### 1.A Single Layer Network & Logistic Regression

The central paradigm of machine learning is that there is some true function $f(\vec{x}) = \vec{y}$ generating our data. Our objective is to learn a model with tunable parameters that can approximate this true function. Ideally, this model could make new predictions and reveal novel insights on the data. For small molecules, $\vec{x}$ is a representation of such a molecule, and $\vec{y}$ are biochemical properties such as solubility, binding, and efficacy in a biological context.

One of the most fundamental and basic model used for this purpose is a Multi-Layer Perceptron (MLP), also known as a feed-forward network (FFN). These networks consist of "fully connected layers" stacked on top of each other with activation functions between the layers.

A single linear layer has operations boil down to basic linear algebra:

$$\vec{y} = \vec{x} \textbf{W} + \vec{b}$$

Here, $\vec{x}$ is a feature vector size $n$, which represents the input to the model:

$$ x = [x_0, x_1, ... x_n] $$

Our goal is to learn the weight matrix $\textbf{W}$ and the bias $\vec{b}$ that transform $\vec{x}$ into $\vec{y}$. When linear layers are stacked on top of each other with activation functions in between, they form what are known as MLPs or FFNs.

#### 1.A.1 Question (1pt)

Assume that the input data $\vec{x}$ is 10-dimensional (i.e there are 10 features per sample), and the output $\vec{y}$ is 3-dimensional, what is the total number of parameters learned in the formulation presented above? Assume it is a single linear layer.

**ANSWER**:



#### 1.A.2 Question (2pt)

So, suppose that we have $f(\vec{x}) = \vec{y}$. The idea is that $f(\vec{x})$ is unknown, but we have many datapoints of $(x_i, y_i)$. Can we find some $\textbf{W}, \textbf{b}$ such that $$\vec{y_i} \approx \vec{x_i} \textbf{W} + \vec{b}$$ for all the datapoints? If we can learn those parameters, we could make predicitons on unseen data.

Finding the best parameters, a process known as training, typically involves optimizing an objective function. Intuitively, this means adjusting the weights and biases so that the loss function, the measure of how far our model's predictions deviate from the actual data, is minimized. In principle, the lower the loss or objective function is, the better the model describes the data. Ideally, the model not only fits the training data well but also generalizes to new, unseen data.

We will use the cross entropy loss:

$$Loss(y_{pred}, y_{true}) = - (y_{true} \cdot log(y_{pred}) + (1 - y_{true}) \cdot log(1 - y_{pred})) $$

Here *y_true* is 1 for a positive example and 0 for negatives, and *y_pred* is a probability for the positive class ranging from 0 to 1. For simpliciy, we will assume that our input $\vec{x}$ is 2-dimensional. In order to obtain a probability (between 0 and 1) we must normalize the output $y_{pred}$. We do so using the sigmoid function:

$$ y^{pred} = sigmoid(\vec{x}W + \vec{b}) = \frac{e^{\vec{x}W + \vec{b}}}{1 + e^{\vec{x}W + \vec{b}}}$$

In order to learn *W* and *b*, we can use gradient descent: we comptue the derivative of the loss with respect to the parameters.

$$
\begin{align}
    \frac{dLoss}{dW} &= (y_{pred} - y_{true})x \\
    \frac{dLoss}{db} &= (y_{pred} - y_{true})
\end{align}
$$

The loss function tells us how bad the model parameters are, and we want to make the model less bad. The gradient gives us an estimate of the correct direction to change the parameters to make the model less bad. After enough iterations, ideally the model is good enough to explain not just training data but also held out data.

Implement the functions below to perform logistic regression on two synthetic datasets. This is important practice before moving to actual small molecules. It may be useful to print the shape of the input to make sure you are handling dimensions correctly.

In [ ]:
# Generate synthetic datasets for binary classification
# Dataset 1: Linearly separable data
# Dataset 2: Data distributed across Gaussian quantiles
# Both datasets consist of 1000 samples with 2 features each
X1, Y1 = make_classification(n_samples=1000, n_features=2, n_redundant=0, n_repeated=0, class_sep=5.0, n_classes=2, random_state=1)
Y1 = Y1[..., None]
X2, Y2 = make_gaussian_quantiles(n_samples=1000, n_features=2, n_classes=2, random_state=1)
Y2 = Y2[..., None]

def sigmoid(x):
    """Sigmoid activation of the input x."""
    # COMPLETE HERE (hint: use np.exp)
    # Output should have the same shape as the input (num_samples, 1)
    pass

def predict(x, W, b):
    """Returns y_pred given the input and learned parameters."""
    # COMPLETE HERE (hint: use the sigmoid function above, and np.dot)
    # Output should have shape (num_samples, 1)
    pass

def loss(y_pred, y_true):
    """Returns the cross-entropy loss given the prediction and target."""
    # (hint: use np.log)
    # Consider adding a small epsilon to the log if you run into errors.
    # Return per sample (no averaging in the first dim)
    # Ouput should have shape (num_samples, 1)
    pass

def dLossdW(y_pred, y_true, x):
    """Comptues the derivative of the loss with respect to W."""
    # Return per sample (no averaging in the first dim)
    # Output should have shape (num_samples, 2)
    pass

def dLossdb(y_pred, y_true):
    """Comptues the derivative of the loss with respect to b."""
    # Return per sample (no averaging in the first dim)
    # Output should have shape (num_samples, 1)
    pass

### DON'T MODIFY BELOW

def gradient_descent_solver(x, y_true):
    # Initialize weights
    W = np.array([0.0, 0.0])[:, None]
    b = np.array([0])
    alpha = 1.0
    num_steps = 1000

    # Perform steps of gradient descent
    y_pred = predict(x, W, b)
    L_start = loss(y_pred, y_true).mean()
    accuracy_start = ((y_pred > 0.5) == y_true).mean()

    for _ in range(num_steps):
        y_pred = predict(x, W, b)
        L = loss(y_pred, y_true).mean()
        accuracy = ((y_pred > 0.5) == y_true).mean()

        dW = dLossdW(y_pred, y_true, x)
        db = dLossdb(y_pred, y_true)
        W = W - alpha * dW.mean(axis=0)[:, None]
        b = b - alpha * db.mean(axis=0)

    print("Start loss: ", L_start)
    print("Final loss: ", L)

    print("Start accuracy: ", accuracy_start)
    print("Final accuracy: ", accuracy)
    return W, b

def plot_results(x, y_true, W, b):
    plt.figure()
    plt.scatter(x[:, 0], x[:, 1], c=y_true)

    x1 = np.linspace(-10, 10)
    x2 = 0 * x1 - 0
    plt.plot(x1, x2, c="b", label="Starting boundary")

    x1 = np.linspace(-10, 10)
    x2 = -W[0] / W[1] * x1 - b / W[1]
    plt.plot(x1, x2, c="r", label="Final boundary")

    plt.xlim(-10, 10)
    plt.ylim(-10, 10)
    plt.legend()
    plt.show()

# Run training
W1, b1 = gradient_descent_solver(X1, Y1)
plot_results(X1, Y1, W1, b1)

W2, b2 = gradient_descent_solver(X2, Y2)
plot_results(X2, Y2, W2, b2)

**1.A.3 Question (2pt)**

Comment on the plots above. How did the model perform on the first dataset? How about the second? Why is the model not appropriate for the second dataset?

**ANSWER**:

### 1.B Multi-layer Perceptron (MLP)

Some datasets require more complex models to classify correctly. Let's see if we can improve the performance on the second dataset. This time we will implement a 2-layer MLP or FFN without an activation function. Mathematically, it performs multiplciation with some weights and adds the biases, and does that a second time with some different weight matrix and biases.

$$y = (xW_1 + b_1)W_2 + b_2$$

**1.B.1 Question (1pt)**

Prove that the 2-layer model defined above isn't more powerful than a single layer model. (Hint: can you show the two layer model is still just the same as one layer using some simple algebraic manipulations?)

**ANSWER**:

**1.B.2 Question (3pt)**

In order to increase the modeling power of the model, we introduce a non-linear activation between the two layers. Here we choose the simple ReLU function:

$$ ReLU(x) = max(0, x) $$

There is a mathematical proof of the universal approximation theorem that shows how a two layer FFN with an activation function can model any function given enough training data, compute, and parameters (in practice, this does not work that well).

We now have the following model:

$$y = ReLU(xW_1 + b_1)W_2 + b_2$$

The derivative is now more complex so we are going to use the PyTorch library which automates differentiation for us. All we need to do now is define our model and loss function. Fill out the code below, but now use Pytorch functions and not numpy.



In [ ]:
def relu(x):
    return torch.nn.functional.relu(x)

def sigmoid(x):
    """Sigmoid activation of the input x."""
    # COMPLETE HERE (hint: use torch.exp)
    # Well, there is a built in torch sigmoid function but
    # please do it the hard way for learning purposes.
    pass

def predict(x, W1, b1, W2, b2):
    """Returns y_pred given the input and learned parameters."""
    # COMPLETE HERE (hint: use the sigmoid & relu functions above + torch.mm)
    pass

def loss(y_pred, y_true):
    """Returns the cross-entropy loss given the prediction and target."""
    # COMPLETE HERE (hint: use torch.log)
    # Consider adding a small epsilon to the log if you run into errors.
    # Return per sample (no averaging in the first dim)
    pass

### DON'T MODIFY BELOW

def gradient_descent_solver(x, y_true):
    # Initialize weights
    random = np.random.RandomState(1)
    W1 = random.randn(2, 100) * 0.01
    W2 = random.randn(100, 1) * 0.01
    W1 = torch.nn.Parameter(torch.tensor(W1).float())
    b1 = torch.nn.Parameter(torch.zeros((100,)))
    W2 = torch.nn.Parameter(torch.tensor(W2).float())
    b2 = torch.nn.Parameter(torch.zeros((1,)))
    alpha = 0.1
    num_steps = 1000

    # Perform steps of gradient descent
    x = torch.tensor(x).float()
    y_true = torch.tensor(y_true).float()
    optimizer = torch.optim.SGD([W1, b1, W2, b2], alpha)

    y_pred = predict(x, W1, b1, W2, b2)
    L_start = loss(y_pred, y_true).mean()
    accuracy_start = ((y_pred > 0.5) == y_true).float().mean()

    for _ in range(num_steps):
        optimizer.zero_grad()
        y_pred = predict(x, W1, b1, W2, b2)
        L = loss(y_pred, y_true).mean()
        L.backward()
        optimizer.step()
        accuracy = ((y_pred > 0.5) == y_true).float().mean()

    print("Start loss: ", L_start.item())
    print("Final loss: ", L.item())

    print("Start accuracy: ", accuracy_start.item())
    print("Final accuracy: ", accuracy.item())

# Run training
print("Dataset 1")
gradient_descent_solver(X1, Y1)

print("\nDataset 2")
gradient_descent_solver(X2, Y2)

**1.B.3 Question (1pt)**

How does the model perform now compared to the 1-layer model in the previous problem?

**ANSWER**:

## 2. Modeling Small Molecules

In this problem, you will experiment with various approaches to model small molecules with ML. The fundamental questions are: how can we represent molecules computationally? How can we predict the properties of molecules with these representations?

For this problem, we consider the Tox21 dataset. The Tox21 dataset stores data in the form of **SMILES** strings, which is a standard format of describing chemical species. **Chem.MolFromSmiles** converts SMILEs strings into a representation that can be used by RDKit. Although multiple SMILES representations can depict a single small molecule, an algorithm exists to derive a unique canonical SMILES, ensuring that each molecule is represented by a singular, standardized string. In other words, there is a one to one mapping from small molecules to a cannonical SMILEs.

First, visualize some of the molecules in the dataset using RDKit.

In [ ]:
# Initialize graph transformation for chemical data processing
# SMILESToBigraph: Converts SMILES strings to graph representations (bigraphs), which are undirected graphs.
# node_featurizer: CanonicalAtomFeaturizer() transforms atoms in a molecule to feature vectors based on standard atomic properties.
# edge_featurizer: CanonicalBondFeaturizer() transforms bonds between atoms into feature vectors, capturing bond type and properties.

# Initialize the Tox21 dataset with the SMILESToBigraph transformation.
# Tox21 dataset: Contains chemical structures as SMILES strings along with their biological activities for toxicity prediction.
# The dataset will use the defined graph transformation (smiles_to_g) for preprocessing the SMILES strings into graph data suitable for graph-based machine learning.
smiles_to_g = SMILESToBigraph(
    node_featurizer=CanonicalAtomFeaturizer(),
    edge_featurizer=CanonicalBondFeaturizer()
)
Tox21 = Tox21(smiles_to_g)

In [ ]:
# Example 1
print(Tox21[0][0])
Chem.MolFromSmiles(Tox21[0][0])

In [ ]:
# Example 2
caffeine_smiles = "CN1C=NC2=C1C(=O)N(C(=O)N2C)C"
Chem.MolFromSmiles(caffeine_smiles)

After viewing the images of the the two examples, please take a look at this guide explaining various functional groups: https://www.masterorganicchemistry.com/2010/10/06/functional-groups-organic-chemistry/. In chemistry, a functional group is a particular arrangement of atoms and their bonds that have predictable chemical properties.

**2.A.1 Question**

Can you identify a functional group in Example 1 and another functional group in Example 2? In very general terms, what are some properties of the functional groups you identified?

**ANSWER:**

### 2.A MLP Over Fingerprints

A Morgan fingerprint is a way of converting a chemical representation into a binary representation that respects chemical properties. As a result, a Morgan fingerprint can be used for simularity search, clustering, or as input to a deep learning model.

We will train our first model using a simple MLP over Morgan Fingerprints. You can find more information about it here: https://www.rdkit.org/docs/GettingStartedInPython.html or by reviewing lecture slides.

In [ ]:
fpgen = AllChem.GetMorganGenerator(radius=3)
mol = Chem.MolFromSmiles(Tox21[0][0])
ao = AllChem.AdditionalOutput()
ao.CollectBitInfoMap()
fp = fpgen.GetCountFingerprint(mol, additionalOutput=ao)
arr = np.zeros((0,), dtype=np.int8)
Chem.DataStructs.ConvertToNumpyArray(fp, arr)
print("Number of bits: ", len(arr))
print("Total non zero entries:", (arr > 0).sum())
print("Maximum count:", arr.max())

The fingerprint is a count vector of length 2048, where each entry corresponds to a substrucuture and the number of times it appears in the molecule.

**2.A.2 Question (1pt)**

Find the most represented bit (maximum count in the array) and visualize it.  Where in the molecule does this pattern appear?

In [ ]:
def most_represented_bit(arr):
    # COMPLETE HERE
    pass

# Change num_item to any value < count for that bit
num_item = 0
bi = ao.GetBitInfoMap()
idx = most_represented_bit(arr)
Chem.Draw.DrawMorganBit(mol, idx, bi, whichExample=num_item)

**ANSWER**:

**2.A.3 Question (2pt)**

We process the Tox21 dataset so that we have the fingerprints for all molecules, and split the data randomly into training and testing. The Tox21 contains labels for various properties of these small molecules, and our goal is to predict these properties. We predict 12 labels for each molecule.

Because the positive to negative ratio is imbalanced, we measure performance using the ROC-AUC (instead of accuracy) for each label individually. ROC-AUC measures the area under the curve that plots True Positive Rate against False Positive Rate. This metric is more comprehensive for assessing the quality of a model than simpler metrics.

Implement a small MLP and train it. Try different values for the hidden dimension, fingerprint radius and dropout. Compare results and comment. Try at least 2 other combinations of hyperparameters.

In [ ]:
# COMPLETE HERE (i.e. play with different values)
RADIUS = 3
HIDDEN_DIM = 128
DROPOUT = 0
NUM_EPOCHS = 10
FP_SIZE = 2048

class MLP(torch.nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim):
        super().__init__()
        self.w1 = torch.nn.Linear(in_dim, hidden_dim)
        self.relu = torch.nn.ReLU()
        self.dropout = torch.nn.Dropout(DROPOUT)
        self.w2 = torch.nn.Linear(hidden_dim, out_dim)

    def forward(self, batch):
        data = batch["data"]
        # COMPLETE HERE (hint: each pytorch layer can be called as a function)
        pass

# DON'T MODIFY BELOW
def process_sample(sample):
    smiles = sample[0]
    labels = sample[2]
    mask = sample[3]
    fpgen = AllChem.GetMorganGenerator(radius=RADIUS, fpSize=FP_SIZE)
    mol = Chem.MolFromSmiles(smiles)
    ao = AllChem.AdditionalOutput()
    ao.CollectBitInfoMap()
    fp = fpgen.GetCountFingerprint(mol, additionalOutput=ao)
    arr = np.zeros((0,), dtype=np.int8)
    Chem.DataStructs.ConvertToNumpyArray(fp, arr)
    arr = torch.tensor(arr).float()
    return {"data": arr, "labels": labels, "mask": mask}

def create_dataset():
    dataset = list(map(process_sample, Tox21))
    train, test = train_test_split(dataset, test_size=0.2, random_state=1)
    return train, test

def create_model():
    model = MLP(FP_SIZE, HIDDEN_DIM, 12)
    return model

def evaluate(model, dataloader):
    out_pred, out_labels, out_mask = [], [], []
    for batch in dataloader:
        mask = batch["mask"]
        labels = batch["labels"]
        y_pred = model(batch).sigmoid()
        out_pred.append(y_pred)
        out_labels.append(labels)
        out_mask.append(mask)

    out_pred = torch.cat(out_pred).detach().numpy()
    out_labels = torch.cat(out_labels).detach().numpy()
    out_mask = torch.cat(out_mask).bool().detach().numpy()

    aucs = []
    for i in range(12):
        preds = out_pred[:, i]
        labels = out_labels[:, i]
        mask = out_mask[:, i]
        preds = preds[mask]
        labels = labels[mask]
        aucs.append(roc_auc_score(labels, preds))

    return np.mean(aucs)

def train(model, train_dataloader, test_dataloader, num_epochs):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    train_loss = []
    train_aucs = []
    test_aucs = []
    for _ in tqdm(range(num_epochs), total=num_epochs):
        avg_loss = 0
        model.train()
        for batch in train_dataloader:
            optimizer.zero_grad()
            mask = batch["mask"]
            labels = batch["labels"]
            y_pred = model(batch)
            loss = torch.nn.functional.binary_cross_entropy_with_logits(y_pred, labels, reduction="none")
            loss = (loss * mask).sum(dim=1) / mask.sum(dim=1).clamp(min=1)
            loss = loss.mean()
            loss.backward()
            optimizer.step()
            avg_loss += loss.item()

        model.eval()
        with torch.no_grad():
            train_auc = evaluate(model, train_dataloader)
            test_auc = evaluate(model, test_dataloader)

        avg_loss /= len(train_dataloader)
        train_loss.append(avg_loss)
        train_aucs.append(train_auc)
        test_aucs.append(test_auc)

    return train_loss, train_aucs, test_aucs

train_set, test_set = create_dataset()
train_dl = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True)
test_dl = torch.utils.data.DataLoader(test_set, batch_size=32, shuffle=False)
model = create_model()
train_loss, train_aucs, test_aucs = train(model, train_dl, test_dl, num_epochs=NUM_EPOCHS)

fig, (ax1, ax2, ax3) = plt.subplots(3)
ax1.plot(train_loss)
ax1.set_title("Training Loss")
ax2.plot(train_aucs)
ax2.set_title("Training ROC-AUC")
ax3.plot(test_aucs)
ax3.set_title("Test ROC-AUC")

print("Final Training ROC-AUC: ", train_aucs[-1])
print("Best Training ROC-AUC: ", max(train_aucs))
print("\nFinal Testing ROC-AUC: ", test_aucs[-1])
print("Best Testing ROC-AUC: ", max(test_aucs))

**2.A.4 Question (1pt)**

How does the radius affect performance? How does the size of the hidden dimension affect performance?

**ANSWER**

**2.A.5 Question (1pt)**

Why does the test AUC initially increase with training but then starts to decline as training time increases? Try increasing the dropout. Does this phenomenon still occur?

**ANSWER**

### 2.B RNN over Smiles

We now shift our attention to using reccurrent neural network (RNN) over smile strings. RNN's are used to encode sequences. At each step an RNN takes as input it's current state *h* and the input at that step *x*:

$$ h_i = f(x_i, h_{i-1}) $$

Consider the simplest example possible, where we apply a simple linear layer on the concatenated input and state vectors.

$$ h_i = W [x_i, h_{i-1}] + b $$

**2.B.1 Question (1pt)**

This formulation, without activation functions, leads to exploding / vanishing gradients. Provide some intuition as to why that might be the case. (Hint: consider what happens to W as you start to unroll the computation across multiple steps, what happens to the eigenvalues of W? What does it imply when they are greater than 1, or lesser than 1)?

**ANSWER**:

In order to address this challenge, and improve RNN's capacity to model long sequences, LSTM's were introduced. Without going in too much detail, LSTMs control the flow of information using learned input and output gates. This has been shown to be an effective way to model longer sequences. You can read more about them here: https://colah.github.io/posts/2015-08-Understanding-LSTMs/

**2.B.2 Question (3pt)**

You will now implement an RNN / LSTM. Fill out the missing code below, here again play with some of the values for the hidden dimension and dropout. Comment on your observations. This make take a bit of time to train, so no need to try too many things.

In [ ]:
# COMPLETE HERE (i.e. modify to a different value)
HIDDEN_DIM = 100
DROPOUT = 0
NUM_EPOCHS = 10

class RNN(torch.nn.Module):

    def __init__(self, in_dim, hidden_dim, out_dim):
        super().__init__()
        self.embedding = torch.nn.Embedding(in_dim, hidden_dim)
        self.lstm = torch.nn.LSTM(
            hidden_dim,
            hidden_dim,
            dropout=DROPOUT,
            batch_first=True,
            bidirectional=True
        )
        self.fc = torch.nn.Linear(2 * hidden_dim, out_dim)

    def forward(self, batch):
        data = batch["data"]
        pad_mask = batch["pad_mask"]
        max_len = data.shape[1]

        # COMPLETE HERE

        # First we embed each input token into a vector
        # Use the self.embedding layer
        emb = ...

        # Compute lengths from padding mask & use the above
        lengths = ...

        # In order to ignore padding we use
        out = torch.nn.utils.rnn.pack_padded_sequence(emb, lengths=lengths, batch_first=True, enforce_sorted=False)

        # Now we pass it to the LSTM (which outputs out, state). Ignore the state.
        out = ...

        # Now we unpack, we use
        out = torch.nn.utils.rnn.pad_packed_sequence(out, batch_first=True, total_length=max_len)[0]

        # Compute the average vector for the sequence
        # Beware of padding! Use the mask! Note that you will need to
        # expand the mask to a 3D Tensor. You can do so with mask.unsqueeze(-1)
        # you will also need to unsqueeze the lengths when dividing by them to take the average
        out = ...

        # Finally apply the fc layer
        out = ...
        return out


# DON'T MODIFY THIS
vocab = {"~": 0}
def process_sample(sample, max_length):
    smiles = sample[0]
    labels = sample[2]
    mask = sample[3]

    tok_ids = []
    for token in smiles:
        if token not in vocab:
            vocab[token] = len(vocab)
            tok_id = len(vocab)
        else:
            tok_id = vocab[token]
        tok_ids.append(tok_id)

    arr = torch.tensor(tok_ids).long()
    return {"data": arr, "labels": labels, "mask": mask}

def create_dataset():
    max_length = max(len(x[0]) for x in Tox21)
    dataset = list(map(lambda x: process_sample(x, max_length), Tox21))
    train, test = train_test_split(dataset, test_size=0.2, random_state=1)
    return train, test

def create_model():
    return RNN(len(vocab) + 1, HIDDEN_DIM, 12)

def collate_fn(data):

    tok_ids = [d["data"] for d in data]
    pad_mask = [torch.ones_like(d["data"]) for d in data]
    labels = [d["labels"] for d in data]
    mask = [d["mask"] for d in data]

    tok_ids = torch.nn.utils.rnn.pad_sequence(tok_ids, batch_first=True)
    pad_mask = torch.nn.utils.rnn.pad_sequence(pad_mask, batch_first=True)
    labels = torch.stack(labels)
    mask = torch.stack(mask)

    return {"data": tok_ids, "labels": labels, "mask": mask, "pad_mask": pad_mask}


train_set, test_set = create_dataset()
model = create_model()
train_dl = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True, collate_fn=collate_fn)
test_dl = torch.utils.data.DataLoader(test_set, batch_size=32, shuffle=False, collate_fn=collate_fn)
train_loss, train_aucs, test_aucs = train(model, train_dl, test_dl, num_epochs=NUM_EPOCHS)

fig, (ax1, ax2, ax3) = plt.subplots(3)
ax1.plot(train_loss)
ax1.set_title("Training Loss")
ax2.plot(train_aucs)
ax2.set_title("Training ROC-AUC")
ax3.plot(test_aucs)
ax3.set_title("Test ROC-AUC")

print("Final Training ROC-AUC: ", train_aucs[-1])
print("Best Training ROC-AUC: ", max(train_aucs))
print("\nFinal Testing ROC-AUC: ", test_aucs[-1])
print("Best Testing ROC-AUC: ", max(test_aucs))

**2.B.3 Question (1pt)**

How does the performance compare to the MLP baseline? Can you further improve it by modifying the hyperparameters? Try at least 2 other combinations of hyperparameters.

**ANSWER**:

### 2.C GNN over Molecular Representations

Moving on, we will now use the graph representation and a simple message passing neural network (MPNN). The input to GNNs generally, including MPNN, is a graph of $n$ nodes with up to $n^2$ edges. Small molecules are represents with atoms as nodes and bonds as edges.

In general, graphs (including small molecules) are represented such that:

1. The nodes are represented as a matrix of $n \times h$, where each node is a $1 \times h$ vector. So a graph's nodes is a list of 1-D vectors, or equivalently a 2D matrix.

2. The edges are encoded into an adjacency matrix $M$ of shape $n \times n$. $M[i][j] = 1$ when there is an edge from $i$ to $j$.

3. Edges themselves have a vector representation encoding information about the edge. For every entry in $M[i][j]=1$, we create a 1_D vector $e_{ij}$ representing useful features of the edge. Then, we make a matrix $E = [e_{ij},...]$ containing an $e_{ij}$ representation for every edge.

With such inputs, the MPNN does the following computations in a single layer:

$$
\begin{align*}
m_{ij} &= MLP([h_i, h_j, e_{ij}]) \\
h_i^{neigh} &= \sum_j m_{ij} \\
h_i^{new} &= MLP(h_i, h_i^{neigh})
\end{align*}
$$

The first equation is saying that for every edge, $e_{ij}$, we calculate a message by taking that edge and the two nodes it connects, and feeding it into an MLP. The output is a message $m_{ij}$

Once we have all the messages $m_{ij}$, we calculating a new hidden representation called $h_i^{neigh}$ which works by taking a node $i$ and summing all the messages if that message came from an edge that touched that node.

Lastly, the new hidden representation is an MLP that takes in $h_i^{neigh}$ and the old $h_i$ and its output is $h_i^{new}$ which updates the hidden representation for each node.

Since this is a single layer, this can be done iteratively using many layers. Also, in the simple MPNN, edge representations $e_{ij}$ are not iteratively updated in a forward pass. But, it is possible, for instance, to make an embedding layer to generate edge representations, such that they can be updated during training.

**2.C.1 Question (1pt)**

For a given molecule, how many message passing steps would you need for every atom to "see" every other atom?

**ANSWER**:

**2.C.2 Question (3pt)**

Again complete the missing code and try a couple of different hyperparameters.

In [ ]:
# COMPLETE HERE (i.e. modify to a different value)
HIDDEN_DIM = 64
NUM_STEPS = 4
DROPOUT = 0
EPOCHS = 20


class GNNLayer(torch.nn.Module):

    def __init__(self, dim, dropout):
        super().__init__()
        self.message_mlp = torch.nn.Sequential(
            torch.nn.Linear(dim * 3, dim),
            torch.nn.SiLU(),
            torch.nn.Dropout(dropout),
            torch.nn.Linear(dim, dim)
        )
        self.node_mlp = torch.nn.Sequential(
            torch.nn.Linear(dim * 2, dim),
            torch.nn.SiLU(),
            torch.nn.Dropout(dropout),
            torch.nn.Linear(dim, dim)
        )
    def message(self, edges):
        node_src = edges.src['h']
        node_dst = edges.dst['h']
        edge = edges.data['e']

        # COMPLETE HERE (hint: use torch.cat in dim=-1 to concatenate and apply the message_mlp)
        msg = ...
        return {'msg_h': msg}

    def forward(self, graph, nodes, edges):
        with graph.local_scope():
            # node feature
            graph.ndata['h'] = nodes

            # edge feature
            graph.edata['e'] = edges

            # Compute messages
            graph.apply_edges(self.message)
            graph.update_all(dgl.function.copy_e('msg_h', 'm'), dgl.function.sum('m', 'h_neigh'))
            h_neigh = graph.ndata['h_neigh']

            # Compute node updates
            # COMPLETE HERE (hint: use torch.cat in dim=-1 to concatenate and apply the node mlp)
            h_new = ...
            return h_new


# DON'T MODIFY THIS
class GNN(torch.nn.Module):
    def __init__(self, dim, dropout):
        super().__init__()
        self.node_fc = torch.nn.Linear(74, dim)
        self.edge_fc = torch.nn.Linear(12, dim)
        self.gnn = GNNLayer(dim, dropout)
        self.fc = torch.nn.Linear(dim, 12)

    def forward(self, batch):
        g = batch["graph"]
        nodes = self.node_fc(g.ndata["h"])
        edges = self.edge_fc(g.edata["e"])

        for i in range(NUM_STEPS):
            # We add a residual connection with helps with stability
            nodes_new = self.gnn(g, nodes, edges)
            nodes = nodes + torch.nn.functional.relu(nodes_new)

        g.ndata["h_out"] = nodes
        out = dgl.mean_nodes(g, "h_out")
        out = self.fc(out)
        return out

def process_sample(sample):
    return {"graph": sample[1], "labels": sample[2], "mask": sample[3]}

def create_dataset():
    dataset = list(map(process_sample, Tox21))
    train, test = train_test_split(dataset, test_size=0.2, random_state=1)
    return train, test

def create_model():
    return GNN(HIDDEN_DIM, DROPOUT)

def collate_fn(data):
    graph = dgl.batch([d["graph"] for d in data])
    labels = torch.stack([d["labels"] for d in data])
    mask = torch.stack([d["mask"] for d in data])
    return {"graph": graph, "labels": labels, "mask": mask}


train_set, test_set = create_dataset()
model = create_model()
train_dl = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True, collate_fn=collate_fn)
test_dl = torch.utils.data.DataLoader(test_set, batch_size=32, shuffle=False, collate_fn=collate_fn)
train_loss, train_aucs, test_aucs = train(model, train_dl, test_dl, num_epochs=EPOCHS)

fig, (ax1, ax2, ax3) = plt.subplots(3)
ax1.plot(train_loss)
ax1.set_title("Training Loss")
ax2.plot(train_aucs)
ax2.set_title("Training ROC-AUC")
ax3.plot(test_aucs)
ax3.set_title("Test ROC-AUC")

print("Final Training ROC-AUC: ", train_aucs[-1])
print("Best Training ROC-AUC: ", max(train_aucs))
print("\nFinal Testing ROC-AUC: ", test_aucs[-1])
print("Best Testing ROC-AUC: ", max(test_aucs))

**2.C.3 Question (1pt)**

How does the performance compare to the MLP and RNN baselines? Can you improve it by playing with hyperparameters (you still get full credit regardless)? Try at least 2 other combinations of hyperparameters.

**ANSWER**:

## 3. Molecular Docking


In lecture we studied the task of molecular docking: given a protein target's structure in 3D, we wish to geometrically model the 3D coordinates of a binding partner. In this problem, we first review the question of symmetry preserving models, which is particuarly important to 3D tasks such as molecular docking. Then, we will run a few docking predictions using DiffDock (https://arxiv.org/abs/2210.01776) and analyze them.

### 3.A Invariance & Equivariance

Because the docking task is inherently a 3D task (i.e we are predicting coordinates), it's important to think about the geometrical symmetries that we wish our model to handle gracefully. We call this family of models "invariant" or "equivariant" to a group transformation. This is particularly neat property that significantly reduces the space of functions that the model considers, helping optimization and generalization.

#### 3.A.1 Question

What does it mean for a model to be invariant to translations and rotations of the input? How about equivariant? Give a simple mathematical definition using `f` as the model function, `x` as the input, `R` as a rotation matrix, and `t` as a translation vector.

**ANSWER**:

#### 3.A.2 Question

A convolutional neural network has a particular invariant property that makes it suited for certain data types. Explain.

**ANSWER**:

#### 3.A.3 Question

There are many ways to achieve invariance / equivariance, and it is currently a very active area of research. Here, we will study the the Equivariant Graph Neural Network (EGNN)  model by Satoras et. al: (https://arxiv.org/pdf/2102.09844.pdf).

Recall the GNN layer with node features `h` and edge features `e`. We compute messages for each edge, aggregate the messages and used the result to update the features. We modify the formulas a little bit:

$$ \begin{align} m_{ij} &= MLP(h_i, h_j, e_{ij}) \\
 m_i &= \frac{1}{N} \sum_j m_{ij} \\
 \tilde{h_i} &= MLP(h_i, m_i) \end{align}$$


An EGNN layer is very similar, defined by the following set of equations, but takes in new inputs $x$ which represent the 3-D (or 2-D or 1-D) spatial coordinates of the input. Please note that small molecules are not flat, and have interesting geometries. For an extreme example, take a look at buckminsterfullerene.


Given a set of input features $h$ and coordinates $x$:

$$ \begin{align} m_{ij} &= MLP(h_i, h_j, e_{ij}, ||x_i - x_j||^2) \\
 m_i &= \frac{1}{N} \sum_j m_{ij} \\
h_{new} &= MLP(h_i, m_i) \\
x_{new} &= x_i + \frac{1}{N} \sum_j MLP(m_{ij}) \cdot (\vec{x_i}- \vec{x_j}) \end{align}$$


The only changes are the use of pairwise distances in the first equation and the coordinate update in the last equation.

Prove that EGNN is indeed equivariant to translation and rotation by showing that applying a rotation `R` and translation `t` to the input `x_i` and `x_j` is equivalent to applying it to the output `x_new`. Please mathematically typeset the solution using LaTex markup.


**ANSWER**:

#### 3.A.4 Question

We will now implement the EGNN and verify its equivariance. We are not training the model on any dataset, but we have a test function to make sure that the implementation is actually equivariant.

In [ ]:
# HELPERS
class MLP(object):
  """A random two layer neural network."""

  def __init__(self, in_dim, out_dim):
    self.w1 = np.random.randn(in_dim, out_dim)
    self.b1 = np.random.randn(out_dim)
    self.w2 = np.random.randn(out_dim, out_dim)
    self.b2 = np.random.randn(out_dim)

  def forward(self, h):
    h_new = np.dot(h, self.w1) + self.b1
    h_new = h_new * (h_new > 0)
    h_new = np.dot(h_new, self.w2) + self.b2
    return h_new

  def __call__(self, h):
    return self.forward(h)

def apply_transform(R, t, x):
  """Apply a rotation and translation to a set of points."""
  return np.dot(x, R.T) + t


# COMPLETE BELOW
class EGNN(object):
  def __init__(self, dim):
    """Random weight initialization."""
    self.message_mlp = MLP(dim * 3 + 1, dim)
    self.feature_mlp = MLP(dim * 2, dim)
    self.coords_mlp = MLP(dim, 1)

  def forward(self, h, e, x):
    """Apply an EGNN layer.

    Parameters
    ----------
    h:
      node features as a numpy array of shape (N, D)
    e:
      edge features as a numpy array of shape (N, N, D)
    x:
      set of points in 3D as a numpy array of shape (N, 3)

    Returns
    -------
    h_new:
      new node features as a numpy array of shape (N, D)
    x_new:
      new set of points in 3D as a numpy array of shape (N, 3)

    """
    # COMPLETE HERE

    # Expand h to get all pairs of h's, your final dimension should be (N, N, 2 * D)
    # You can expand with dummy dimensions and then use np.repeat

    # Get all x_i - x_j vectors (you can just broadcast, without repeat)

    # Compute the squared norm of the x_ij vectors

    # Concatenate h pairs, distances, and edge features

    # Compute messages for all i, j pairs

    # Aggregate messages per node (sum across j)

    # Update features

    # Update coordinates
    pass


# DO NOT MODIFY BELOW
def test_equivariance():
  # Set parameters
  N = 64
  D = 32
  h = np.random.randn(N, D)
  e = np.random.randn(N, N, D)
  x = np.random.randn(N, 3)
  egnn = EGNN(D)

  # Sample a random roto-translation
  R = Rotation.random().as_matrix()
  t = np.random.randn(3)
  x_rot = apply_transform(R, t, x)

  # Run model on x
  _, x_new = egnn.forward(h, e, x)
  x_new_rot = apply_transform(R, t, x_new)

  # Run the model on x_rot
  _, x_rot_new = egnn.forward(h, e, x_rot)

  # Compare the outputs
  if np.linalg.norm(x_new_rot - x_rot_new, axis=-1).max() < 1e-4:
    print("Equivariance test passed!")
  else:
    print("Equivariance test failed!")


test_equivariance()

## 4. DiffDock

We will use a state of the art deep learning tool, DiffDock, to produce candidate docking structures. We will analyze the results to get some insight into the inner workings of the model. DiffDock also uses a 3D equivariant model, though it is not the EGNN introduced above but a tensor field network. These are more complex, and we will not study them in this course. If you are interested, you may read more about them here:
https://arxiv.org/abs/1802.08219. For this problem, we will instead focus on a different charachteristic of DiffDock: the "diffusion" process.

In [ ]:
!wget -c -O diffdock.zip https://www.dropbox.com/scl/fi/e23wyl4bfolckjaaablhc/diffdock.zip?rlkey=80p9a46yivyc3jdi7jnq7bnx9&dl=0
!unzip -o diffdock.zip -d diffdock

#### 4.A.1 Question

DiffDock is a specific type of generative model, namely a diffusion-based model. These models are trained to iteratively reverse a noise distribution (ex: gaussian) to the data distribution. Due to their generative nature, they can be sampled from, and every sample requires a certain number of denoising steps. For more info, see this very good blog post: https://lilianweng.github.io/posts/2021-07-11-diffusion-models/

Interestingly, duffision models's loss functions are mathematically very similar to those used for training VAEs, despite looking different on the surface level: https://proceedings.neurips.cc/paper_files/paper/2023/hash/ce79fbf9baef726645bc2337abb0ade2-Abstract-Conference.html

You are provided with pre-computed structures from DiffDock for this problem. If you are interested in making your own predictions you may use this colab notebook: https://colab.research.google.com/github/hgbrian/biocolabs/blob/master/DiffDock.ipynb


In [ ]:
# RUN THIS
DIFFDOCK_DATA = "/content/diffdock/diffdock/"
GT_PATH = "6o5g_B_LMJ.sdf"

ground_truth = os.path.join(DIFFDOCK_DATA, GT_PATH)
predictions = []
for file_name in os.listdir(DIFFDOCK_DATA):
  if "rank" not in file_name:
    continue
  else:
    meta = file_name.split("_")
    group = meta[0]
    rank = int(meta[2][-1])
    confidence = float(meta[3].split("confidence")[1][:-4])
    predictions.append({
        "path": os.path.join(DIFFDOCK_DATA, file_name),
        "group": group,
        "rank": rank,
        "confidence": confidence
    })

print("Ground truth path:\n")
print(ground_truth)

print("\nPredictions:")
print(pd.DataFrame(predictions))

The table consists of 15 entries, each representing a different prediction made by the DiffDock model. These predictions are categorized based on three different sets of denoising steps: 5, 10, and 20. For each set of denoising steps, the model generates 5 distinct docking predictions, leading to a total of 15 unique prediction entries.

For each of the 3 groups (i.e 5, 10 and 20), report the best RMSD across the 5 samples. How did performance change as a function of the number of denoising steps?

In [ ]:
# HELPERS
def load_coordinates_sdf(path):
    """Load molecule from sdf."""
    suppl = Chem.SDMolSupplier(path)
    mol = next(suppl)
    return mol

def compute_rmsd(mol1, mol2):
    """Compute the RMSD between mol1 and mol2."""
    return Chem.rdMolAlign.CalcRMS(mol1, mol2)

# COMPLETE HERE

**ANSWER:**

#### 4.A.2 Question

Perform the same analysis, this time comparing strategies for measuring performance across the 5 samples:

- The mean RMSD (same as before)
- The best RMSD of the 5 samples
- The highest confidence ranked of the 5 samples

Comment on your observations.

**ANSWER:**

#### 4.A.3 Question

Visualize the worst and the best prediction (according to the RMSD) using the Molstar visualizer (https://molstar.org/viewer/).

You can just drag and drop the files directly on your browser, super easy. Make sure to visualize both the sdf file (molecule prediction) and the ground truth PDB (target protein + correct molecule pose). Take screenshots for tbe best and worst predictions and display them here.

In [ ]:
# Set the best file and download it by running this cell
# Do not include the /content/diffdock/diffdock prefix
best_prediction = ""
worst_prediction = ""

# Download
from google.colab import files

DIFFDOCK_DATA = "/content/diffdock/diffdock/"
files.download(os.path.join(DIFFDOCK_DATA, best_prediction))
files.download(os.path.join(DIFFDOCK_DATA, worst_prediction))
files.download(os.path.join(DIFFDOCK_DATA, "6O5G.pdb"))

Do you notice anything interesting about the "bad" prediction?

**ANSWER:**

## 5. Genetic Association Studies and Finding eQTLs

Genome-Wide Association Studies (GWAS) utilize genetic information, often derived from whole genome sequencing, to identify single nucleotide polymorphisms (SNPs) associated with various phenotypes. One notable success of GWAS was the identification of genetic variants in the CFH gene that significantly increase the risk of developing age-related macular degeneration (AMD), a leading cause of blindness in older adults.

Beyond GWAS, genetic association studies can also examine how certain SNPs influence variations in gene expression, as observed in RNA sequencing (RNA-seq) experiments. These SNPs may reside within genomic regulatory regions, such as transcription factor binding sites or promoters, affecting gene expression levels directly.

However, it's important to approach these findings with a high degree of scientific skepticism. Not all SNPs linked to variations in RNA-seq data are necessarily functional; some associations may be spurious. Rigorous validation and replication are essential to confirm the role of these SNPs as eQTLs—genetic variants that explain variations in RNA expression.

In the final problem set question, we will explore how variations in gene expression, measured through RNA-seq, can differentiate subpopulations within a broader population. We will identify eQTLs that account for some of this variation in gene expression.

Please download Expdata.txt and SNPdata.txt from https://www.dropbox.com/scl/fo/soyclczbf0p4v1n7x0tnf/ALOOX7QjrI-e1BKqSFr6MQQ?rlkey=kddmo48uctnyu3jgn8xx4016t&e=2&dl=0

Upload it to this notebook and work on the files.

In [ ]:
!wget -c "https://www.dropbox.com/scl/fi/tjch877x78pglm0cqfuso/eQTLs.zip?rlkey=6tw9gotl098o8kshg67myfkwo&st=bnldxdln&dl=0" -O eQTLs.zip
!unzip -o eQTLs.zip -d eqtl_data
os.unlink("eQTLs.zip")
eqtl_data = {}
for fname in os.listdir("eqtl_data"):
    with open(os.path.join("eqtl_data", fname), "r") as R:
        eqtl_data[fname] = R.read().encode('utf-8')

--2024-10-26 05:58:17--  https://www.dropbox.com/scl/fi/tjch877x78pglm0cqfuso/eQTLs.zip?rlkey=6tw9gotl098o8kshg67myfkwo&st=bnldxdln&dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc5d77f62559c31ed44fab3b40bf.dl.dropboxusercontent.com/cd/0/inline/CdJst1GaMEQkfWueAetTb7_N_ZT91hCiUqzvmzXh9xCWvlKjll7mcEHd2zsYKFNJxbm4p2vsztcf1-9noSCQEjXIXnzm6GKXuvvL8Va7SUruUDLpa5EgsIiuxeRBhI0BzybckHXU9wIgcbnuskoue3zQ/file# [following]
--2024-10-26 05:58:17--  https://uc5d77f62559c31ed44fab3b40bf.dl.dropboxusercontent.com/cd/0/inline/CdJst1GaMEQkfWueAetTb7_N_ZT91hCiUqzvmzXh9xCWvlKjll7mcEHd2zsYKFNJxbm4p2vsztcf1-9noSCQEjXIXnzm6GKXuvvL8Va7SUruUDLpa5EgsIiuxeRBhI0BzybckHXU9wIgcbnuskoue3zQ/file
Resolving uc5d77f62559c31ed44fab3b40bf.dl.dropboxusercontent.com (uc5d77f62559c31ed44fab3b40bf.dl.dropboxusercontent.com)... 162.12

You are free to use the following function to help you process the raw `ExpData.txt` and `SnpData.txt` files.

In [ ]:
def process_file(inp, mode):
  """
  Processes a simplified representation of genetic or gene expression data from *Data.txt files.

  This function is designed for educational purposes or small datasets and does not handle
  the complexity typically associated with real-world genomic data formats like VCF or BAM.
  It assumes input data is structured in a basic text format with space-separated values, where
  the first line contains headers (traits or gene identifiers) and subsequent lines contain
  numerical data representing gene expression levels or SNP values for each sample.

  Parameters:
      inp (bytes): The input data in bytes format, typically read from a file.
      mode (str): The processing mode - 'patient' returns data by samples, 'trait' returns data by gene/trait.

  Returns:
      list: If mode is 'patient', returns a list of lists with each sublist containing
      floats representing gene expression levels for each patient.
      dict: If mode is 'trait', returns a dictionary where keys are traits or gene
      identifiers and values are lists of floats representing gene expression levels across samples.

  Example:
      Input example:
          Patient Trait1 Trait2 Trait3
          A 0.1 0.2 0.3
          B 0.4 0.5 0.6
          C 0.7 0.8 0.9

      Usage:
          process_file(inp, "patient")
              Returns: [[0.1, 0.2, 0.3], [0.4, 0.5, 0.6], [0.7, 0.8, 0.9]]
          process_file(inp, "trait")
              Returns: {"Trait1": [0.1, 0.4, 0.7], "Trait2": [0.2, 0.5, 0.8], "Trait3": [0.3, 0.6, 0.9]}
  """
  rows = inp.decode("utf-8").split("\n")
  if mode == "patient":
      return [[float(val) for val in row.split()[1:]] for row in rows[1:] if row]
  elif mode == "trait":
      res = pd.DataFrame([row.split()[1:] for row in rows if row])
      res.columns = res.iloc[0]
      return {k: [float(val) for val in v.values()] for k, v in res[1:].to_dict().items()}

#process_file(eqtl_data["SnpData.txt"], "patient")
#process_file(eqtl_data["SnpData.txt"], "trait")
#process_file(eqtl_data["ExpData.txt"], "patient")
#process_file(eqtl_data["ExpData.txt"], "trait")

**5.A.1**

The `ExpData.txt` file contains log-normalized RNA-seq expression data from our population of 1,000 samples, with 5,000 genes profiled for each sample. Do a principal components analysis on this dataset to find the clusters of samples that have similar patterns of gene expression. Plot the output of your analysis.
In your plots, be sure the axes are labeled with the components you are displaying in each plot. Also make sure that at least one of your plots colours the points corresponding to the samples with the sub-population that you think they should belong to. (Hint: You can modify your $k$-means code from PSET 1 to find these sub-populations. However, it is better to use kmeans packages such as `sklearn.cluster import KMeans`)

You may find the `matplotlib.pyplot` and `sklearn.decomposition.PCA` packages (already imported) to be useful.

In [ ]:
def analyze_expression_data(data):
  """
  After running process_file(..., ...), perform PCA on the data and
  plot the first two principal components. (Since the plot is 2D, we can only
  plot two principal components). Apply clustering as well to see the
  structure of the data. On the plot, please color clusters that you find.
  """
  pass

Describe the patterns that you observe. What is the structure inherent in this population?

**Answer:**

**5.A.2**

The `SnpData.txt` file contains genotyping data for the same 1,000 samples across 500 SNPs. Each SNP's genotype has been called with reference to the same reference genotype; "0" thus represents the reference allele, "2" represents the non-reference allele, and "1" represents a different allele on each strand.

You will find that some of the SNPs (more than 5, less than 100) are eQTLs, that is, they have an effect on the expression of one or more of the genes we collected expression data for. Using whatever model you see fit, search for these eQTLs using the genotyping data and the expression data. **You may not have the computational resources to test all combinations of SNPs and genes, so you should think about smart ways to choose subsets of each to find some eQTLs - you don't have to find all of them!**

For three of the eQTLs you found, present the evidence you have for why you think it is an eQTL, and not just associated with the expression of a gene by chance alone. *Be sure to include plots in your analysis to support your hypothesis, and to thoroughly explain the method you used to find eQTLs.* You can assume that the association between genotype and expression is linear for eQTLs. *Don't forget that you should be correcting for the fact that you are performing multiple significance tests.*

It may be difficult to think of the way to do it from scratch. Here is a general approach: perform PCA on the SNPs data and expression data (please experiment with different numbers of principal components). See if the SNP principal components are correlated with the expression principal components. Once you find pairs of correalated principal components, you want to map these back to the actual genes and SNPs. Then, when you have a short list of top genes and their SNPs, perform linear regression to show how gene expression of those genes can be a function of certain SNPs. There are other approaches that will have the same results as this, but this is the easiest.

In [ ]:
def process_and_pca(data):
    """
    Apply PCA to reduce dimensionality.
    Returns the PCA-transformed data.

    Ideally, the data come from the functions:
    #data = process_file(eqtl_data["SnpData.txt"], "patient")
    # process_file(eqtl_data["SnpData.txt"], "trait")
    # process_file(eqtl_data["ExpData.txt"], "patient")
    # process_file(eqtl_data["ExpData.txt"], "trait")
    """
    pass

def correlation_and_top_pairs(exp_pcs, snp_pcs, top_n):
    """
    Calculate correlation between expression and SNP PCA components, and return the top_n correlated pairs.
    """
    pass

def perform_regression_and_plot(exp_data, snp_data, regression_pairs):
    """
    Perform regression analysis on selected gene-SNP pairs and plot the most significant.
    Adjusts p-values for multiple testing and plots the two most important principal components of SNP data.
    """
    pass

# Example Usage
exp_pcs = pass
snp_pcs = pass

top_corr_pairs = pass
top_pairs = pass

# Perform regression analysis and plot
regression_results = pass

**Answer:**

eQTL 1:

eQTL 2:

eQTL 3:

**5.A.3**

In the above analysis, we were forced to consider all pairs of SNPs and genes to identify eQTLs. What sources of data that have not been provided as part of this problem would have been useful in constraining the amount of such pairs you had to test? For at least two sources:

i. Give a description of what the dataset would look like (i.e. what are the rows and columns of the data matrix? what kinds of values are stored in the matrix?).

ii. Explain how you would use it to filter out pairs of SNPs and genes that are unlikely to be associated with one another.

**Answer**:

Source 1:

i.

ii.

Source 2:

i.

ii.